In [49]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [50]:
def synthetic_data(w,b,num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))
    y = torch.matmul(X,w)+b
    y += torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

In [51]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [52]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [53]:
features.shape

torch.Size([1000, 2])

In [54]:
labels.shape

torch.Size([1000, 1])

In [55]:
for X,y in data_iter(10, features,labels):
    print(X,'\n',y)
    break

tensor([[ 1.2844,  0.2841],
        [ 2.1973, -1.3253],
        [ 2.0058,  2.1325],
        [ 0.8425, -0.0638],
        [ 1.1839,  1.0073],
        [ 1.4181,  1.3691],
        [ 1.4750,  0.1625],
        [ 0.2286,  0.7548],
        [ 0.7417,  0.5950],
        [-0.4007, -0.5429]]) 
 tensor([[ 5.7942],
        [13.1101],
        [ 0.9623],
        [ 6.0913],
        [ 3.1542],
        [ 2.3648],
        [ 6.6182],
        [ 2.0929],
        [ 3.6485],
        [ 5.2589]])


In [56]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [57]:
w

tensor([[ 0.0083],
        [-0.0057]], requires_grad=True)

In [58]:
b

tensor([0.], requires_grad=True)

In [59]:
def linreg(X, w, b):  #@save
    """线性回归模型。"""
    return torch.matmul(X, w) + b

In [60]:
def squared_loss(y_hat, y):  #@save
    """均方损失。"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [61]:
def sgd(params,lr,batch_size):
    with torch.no_grad():
        for param in params:
#             print(param.grad)
            param -= lr * param.grad/batch_size
            param.grad.zero_()

In [62]:
num_epochs = 10
lr = 0.01
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X,y in data_iter(20,features,labels):
        l = loss(net(X,w,b),y)
        l.sum().backward()
        sgd([w,b],lr,20)
#         print(f'epoch{epoch}, w:{w},b:{b}')
    with torch.no_grad():
        train_l = loss(net(features,w,b),labels)
        print(f'epoch:{epoch},loss:{float(train_l.mean())}')

epoch:0,loss:6.05592679977417
epoch:1,loss:2.2460014820098877
epoch:2,loss:0.8338708877563477
epoch:3,loss:0.3099549412727356
epoch:4,loss:0.11531592160463333
epoch:5,loss:0.04297169670462608
epoch:6,loss:0.01605130359530449
epoch:7,loss:0.006020072847604752
epoch:8,loss:0.0022830956149846315
epoch:9,loss:0.0008882099646143615
